# Text segmentation using Hidden Markov Models

**Q1** asks for the value of the π vector of the initial probabilities in a Hidden Markov Model (HMM) with two states. The π vector represents the probability of the system starting in a particular state. In this context, with two states for an email segmentation task (state 1 for the header and state 2 for the body), we are looking for the initial probabilities of the system being in state 1 and state 2.

To solve this, we usually consider the overall structure of the task at hand. Since the task specifies that every email starts with a header (state 1), it's logical to assume that the probability of starting in state 1 is 1 (or 100%) and the probability of starting in state 2 (body) is 0 since an email cannot start with the body.

Therefore, the π vector would be:

$$ π = [1, 0] $$

This means that there is a 100% chance that an email will start with the header (state 1) and a 0% chance of it starting with the body (state 2).

In [1]:
import os
import glob
import numpy as np
import subprocess

In [2]:
ROOT = os.path.abspath('.')

PERL_DIR = os.path.join(ROOT,'PerlScriptAndModel')
RES_DIR = os.path.join(ROOT,'res')
SEGMENTED_DIR = os.path.join(ROOT, 'segmented')

**Q2:** The transition matrix A is:

$$ A = \begin{pmatrix} 0.999218078035812 & 0.000781921964187974 \\ 0 & 1 \end{pmatrix} $$

This matrix shows the probabilities of moving from one state to another. The element $ A(i,j) $ represents the probability of moving from state i to state j.

From the matrix:
- The probability to move from state 1 (header) to state 2 (body) is the off-diagonal element $ A(1,2) $, which is $ 0.000781921964187974 $.
- The probability to remain in state 2 (body), once in state 2, is the diagonal element $ A(2,2) $, which is $ 1 $.

Thus:
- The probability of moving from state 1 to state 2 is approximately $ 0.078\% $.
- The probability to remain in state 2 once it is reached is $ 100\% $.

The higher probability is the one to remain in state 2, which is 1 or $ 100\% $, and the lower probability is to transition from state 1 to state 2, which is approximately $ 0.078\% $.

The reason behind these probabilities can be explained as follows:

- The transition probability from state 1 to state 2 being low suggests that the header of an email is expected to be relatively short compared to the body, and there is only a small chance at each point that the header will end and the body will begin. Once the transition occurs, it does not reverse, which is consistent with the structure of an email (the header comes first and is followed by the body, and there is no going back to the header).
  
- The transition probability to remain in state 2 being $ 1 $ (or $ 100\% $) reflects the fact that once the email transitions to the body, it stays in the body. There are no further transitions back to the header; hence, the email remains in the body state until it ends.

### Coding/Decoding Mails

In [3]:
DATA_DIR = os.path.join(ROOT,'dat')

# Iterate through files and load the text 
def files_iter(data_dir, with_name=False):
    files = glob.glob('{}/*.dat'.format((data_dir)))
    if with_name:
        for f in files:
            # Get the filename 
            if '/' in f:
                name = f.split('/')[-1].split('.')[0]
            else:
                name = f.split('\\')[-1].split('.')[0]
            
            # Return text and name
            yield name, np.loadtxt(f, dtype=int)
    else:
        for f in files :
            # Return text
            yield np.loadtxt(f, dtype=int)

In [4]:
# And we get a generator that will allow us to iterate through the mails
mail_iter = files_iter(DATA_DIR, with_name=True)

**Q3:** In the context of Hidden Markov Models (HMM), the matrix $ B $ refers to the emission or observation probability matrix. It represents the probability of observing a particular output from a given state. 

Given that a mail is represented by a sequence of characters and that $ N $ is the number of different characters, the size of $ B $ would be determined by the number of states and the number of different observations (characters in this case).

Since the HMM for the email segmentation task has two states (state 1 for the header and state 2 for the body), and if we assume $ N $ different characters that can be observed in each state, the size of $ B $ would be 2 x $ N $. 

Therefore, the size of $ B $ is dependent on the number of different characters $ N $ that are considered in the email's content. If that number is not given, we cannot specify the exact dimensions of $ B $ without additional information. If you have the number of different characters $ N $ used to represent the email, then you can determine the size of $ B $ as 2 x $ N $.

### Distribution files

In [5]:
PERL_DIR = os.path.join(ROOT,'PerlScriptAndModel')

# Writing a function to get the probability data
def get_emission_prob(perl_dir):
    p_text_path = os.path.join(perl_dir, 'P.text')
    
    emission_prob = np.loadtxt(p_text_path)
    
    return emission_prob

In [6]:
# Inputs to the Viterbi function
trans = [[0.99921, 0.00078],  # transition probabilities from state 1
        [0, 1]]            # transition probabilities from state 2
emission_prob = get_emission_prob(PERL_DIR)  # the emission probabilities
states = [1, 2]  # the two states of the HMM
start_prob = [1, 0]  # the probability of starting in each state

### To implement:

In [7]:
# Viterbi function
def viterbi(obs, states, start_prob, trans, emission_prob):
    """
        Viterbi Algorithm Implementation

        Keyword arguments:
            - obs: sequence of observation
            - states:list of states
            - start_prob:vector of the initial probabilities
            - trans: transition matrix
            - emission_prob: emission probability matrix
        Returns:
            - seq: sequence of state
    """

    # Avoid underflow: use the logarithm !
    # Avoid 0 in logarithm: use a small constant !
    small = 1e-10
    
    start_prob = np.log(np.array(start_prob) + small)
    trans = np.log(np.array(trans) + small)
    emission_prob = np.log(np.array(emission_prob) + small)
    
    # Number of observations
    T = len(obs)
    # Number of model states
    N = len(states)
    
    # Initialisation
    log_l = np.zeros((T, N)) - np.inf
    bcktr = np.zeros((T, N), dtype=int)
    
    # Viterbi
    
    # Forward loop:
    log_l[0,:] = start_prob + emission_prob[obs[0], :] # Initialize first column with start probabilities
    
    for t in range(1, T):
        for s in range(N):
            # For each state, find the max log probability of the incoming transitions
            transition_probs = log_l[t - 1, :] + trans[:, s] + emission_prob[obs[t], s]

            # Store the max probability and the state that gave that probability
            log_l[t, s] = np.max(transition_probs)
            bcktr[t, s] = np.argmax(transition_probs)

    # Backward loop
    
    # Start with the state that has the highest probability at the last observation
    path = [np.argmax(log_l[:, - 1])]

    for i in range(T - 1, 0, -1):
        path.append(bcktr[i, path[-1]])

    path.reverse()

    # Return the most probable state sequence
    return [i + 1 for i in path]

In [8]:
mails = [mail for mail in files_iter(DATA_DIR)]

# Test Viterbi on some mails that are given in the dat directory (especially mail11.txt to mail30.txt).
print("Mail 10 :")
print(viterbi(mails[11], states, start_prob, trans, emission_prob))

print("Mail 30 :")
print(viterbi(mails[29], states, start_prob, trans, emission_prob))

Mail 10 :
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [9]:
RES_DIR = os.path.join(ROOT,'res')

# Creating a directory to put the result of the viterbi function
if not os.path.exists(RES_DIR):
    os.mkdir(RES_DIR)
    
# Function that will write a viterbi path for a mail in a dedicated result file
def create_viterbi_path_file(mail_name, viterbi_path):
    with open('{}/{}_path.txt'.format(RES_DIR, mail_name), 'w') as f: 
        f.write(''.join([str(c) for c in viterbi_path]))   

In [10]:
# Using our generator, we get the mail names and data
for name_file, data in mail_iter:
    # Find out the viterbi path using viterbi
    viterbi_path = viterbi(data, states, start_prob, trans, emission_prob)

    print("Viterbi path for mail {} : {}".format(name_file, viterbi_path))
    # Put it in the result file
    create_viterbi_path_file(name_file, viterbi_path)

Viterbi path for mail mail1 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

### Visualizing segmentation

In [11]:
def save_segmented_output(mail_name, segmented_text, segmented_dir):
    # Construct the path for the new segmented file
    segmented_file_path = os.path.join(segmented_dir, f"{mail_name}.txt")
    
    # Write the segmented text to the new file
    with open(segmented_file_path, 'w') as file:
        file.write(segmented_text)
    
    print(f"Segmented output saved to {segmented_file_path}")

In [12]:
# Writing a function to go into the directory and execute the perl script "segment.pl" on the mail in the given path
def exec_perl_script(mail, path):
    command = ["perl", "segment.pl", mail, path]
    result = subprocess.run(command, cwd=PERL_DIR, capture_output=True, text=True, check=True)
    return result.stdout

# Writing a function getting the original mail, the result of viterbi, and applying the segmentation script
# Then putting the result

def segment_mail(mail_name, data_dir, output_dir, segmented_dir):
    # Get the full path of the mail
    mail = os.path.join(data_dir, mail_name + '.txt')

    # Get the full path of the result
    path = os.path.join(output_dir, mail_name + '_path.txt')

    # Execute the visualization script
    formatted_mail = exec_perl_script(mail, path)

    # Get the results
    with open(path, 'r') as file:
        formatted_mail_text = file.read()

    # If this was not the last line, return the text cut in to parts: header and body
    if len(formatted_mail) > 0:
        # Find the cutting line
        cut = formatted_mail.find("\n========================== coupez ici ==========================\n")
        
        header = formatted_mail[:cut]
        body = formatted_mail[cut:]

        # Save the segmented output to a new file in the "segmented" folder
        save_segmented_output(mail_name + "_header", header, segmented_dir)
        save_segmented_output(mail_name + "_body", body, segmented_dir)

        # Return the header and the body
        return header, body
    else:
        # Save the segmented output to a new file in the "segmented" folder
        save_segmented_output(mail_name, formatted_mail, segmented_dir)
        
        # If not, it's just a header
        return formatted_mail, None

In [13]:
# Getting mails names
mails = [os.path.basename(f).split('.')[0] for f in glob.glob(os.path.join(DATA_DIR, '*.dat'))]

if not os.path.exists(SEGMENTED_DIR):
    os.mkdir(SEGMENTED_DIR)

# Call the function and look at the result of segmentation
for mail_name in mails:
    header, body = segment_mail(mail_name, DATA_DIR, RES_DIR, SEGMENTED_DIR)

Segmented output saved to c:\SDTSIA214_TP2_HMM\segmented\mail1_header.txt
Segmented output saved to c:\SDTSIA214_TP2_HMM\segmented\mail1_body.txt
Segmented output saved to c:\SDTSIA214_TP2_HMM\segmented\mail10_header.txt
Segmented output saved to c:\SDTSIA214_TP2_HMM\segmented\mail10_body.txt
Segmented output saved to c:\SDTSIA214_TP2_HMM\segmented\mail11_header.txt
Segmented output saved to c:\SDTSIA214_TP2_HMM\segmented\mail11_body.txt
Segmented output saved to c:\SDTSIA214_TP2_HMM\segmented\mail12_header.txt
Segmented output saved to c:\SDTSIA214_TP2_HMM\segmented\mail12_body.txt
Segmented output saved to c:\SDTSIA214_TP2_HMM\segmented\mail13_header.txt
Segmented output saved to c:\SDTSIA214_TP2_HMM\segmented\mail13_body.txt
Segmented output saved to c:\SDTSIA214_TP2_HMM\segmented\mail14_header.txt
Segmented output saved to c:\SDTSIA214_TP2_HMM\segmented\mail14_body.txt
Segmented output saved to c:\SDTSIA214_TP2_HMM\segmented\mail15_header.txt
Segmented output saved to c:\SDTSIA214_

In [14]:
print(header)

1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

In [15]:
print(body)


========================== coupez ici ==========================
 

 Meaningful sentences 
 
 Tracey Lawson 
 
 
 If you ever wanted to look like "one of the most dangerous inmates in prison
 history", as one judge described Charles Bronson, now’s your chance. Bronson -
 the serial hostage taker, not the movie star - has written a health and
 fitness guide in which he shares some of the secrets behind his legendary
 muscle power. 
 
 Solitary Fitness - a title which bears testament to the fact that Bronson, 48,
 has spent 24 of his 28 prison years in solitary confinement - explains how he
 has turned himself into a lean, mean, fitness machine while living 23 hours a
 day in a space just 12 feet by eight feet, on a diet of scrubs grub and at
 virtually no cost. 
 
 The book is aimed at those who want to get fabulously fit without spending a
 fortune on gym memberships, protein supplements or designer trainers, and
 starts with a fierce attack on some of the expensive myths churned out 

**Q4:** To model the problem of segmenting emails into more than two parts using a Hidden Markov Model (HMM), you would increase the number of states to correspond to the different segments you want to identify. In this case, if we want to segment emails into a header, body, and signature, we would have a three-state HMM. The states could be represented as follows:

- State 1: Header
- State 2: Body
- State 3: Signature

Since an email typically flows from the header to the body and then to the signature, the transition matrix $ A $ would represent the probabilities of moving from one state to the next. A likely structure for the transition matrix $ A $ for this three-state model, where each part follows sequentially and there are no transitions backward, would be:

$$ A = \begin{pmatrix} a_{11} & a_{12} & 0 \\ 0 & a_{22} & a_{23} \\ 0 & 0 & a_{33} \end{pmatrix} $$

Where:
- $ a_{11} $ is the probability of staying in the header,
- $ a_{12} $ is the probability of transitioning from the header to the body,
- $ a_{22} $ is the probability of staying in the body,
- $ a_{23} $ is the probability of transitioning from the body to the signature, and
- $ a_{33} $ is the probability of staying in the signature.

Since we know that the process must move forward from the header to the body, and then to the signature, the matrix would look like:

$$ A = \begin{pmatrix} a_{11} & 1 - a_{11} & 0 \\ 0 & a_{22} & 1 - a_{22} \\ 0 & 0 & 1 \end{pmatrix} $$

An example of such a matrix could be:

$$ A = \begin{pmatrix} 0.9 & 0.1 & 0 \\ 0 & 0.95 & 0.05 \\ 0 & 0 & 1 \end{pmatrix} $$

This example assumes:
- There is a 90% chance of staying in the header and a 10% chance of moving to the body from the header.
- Once in the body, there is a 95% chance of staying in the body and a 5% chance of moving to the signature.
- The signature state, once reached, is absorbing, meaning once you enter the signature, you stay there (100% chance).

To model the problem of separating included portions of an email, where these portions always start with the character ">", using a Hidden Markov Model (HMM), you would define at least two states:

- State 1: Not Included (text that is original or not included from previous messages)
- State 2: Included (text that is included from previous messages and marked with ">")

Assuming that the beginning of an included portion is marked by ">", you can design the model such that it transitions to State 2 whenever this character is observed, and it remains in State 1 otherwise.

The transition matrix $ A $ would represent the probabilities of remaining in the current state or moving to the other state. A basic structure for $ A $ could be:

$$ A = \begin{pmatrix} a_{11} & 1 - a_{11} \\ a_{21} & 1 - a_{21} \end{pmatrix} $$

Where:
- $ a_{11} $ is the probability of staying in State 1 given the current state is State 1,
- $ 1 - a_{11} $ is the probability of moving to State 2 from State 1,
- $ a_{21} $ is the probability of moving back to State 1 from State 2,
- $ 1 - a_{21} $ is the probability of staying in State 2.

To accurately model the observation that included portions start with ">", we need an emission probability matrix that accounts for the observation of the ">" character triggering the transition to State 2.

Let's sketch a diagram for this two-state HMM with the described characteristics.

Here's a diagram of a Hidden Markov Model (HMM) with two states aimed at identifying included portions of text in an email. The states "Not Included" and "Included" represent text that is original and text that is included from previous messages, respectively. The diagram highlights the transitions between these states with the associated probabilities, and particularly emphasizes the transition to the "Included" state when the character ">" is observed.